In [ ]:
# https://hub.ultralytics.com/datasets

In [ ]:
# 1. COCO JSON -> YOLO용 txt로 변환
import os
import json
import glob

In [ ]:
def convert_coco_to_yolo(coco_json_folder, output_txt_folder):

    # 폴더를 생성하는 함수
    os.makedirs(output_txt_folder, exist_ok=True)
    # output_txt_folder: 폴더를 생성할 경로를 나타내는 문자열
    # exist_ok: 이 인자는 선택적 매개변수
    #           True : 폴더가 이미 존재하는 경우에는 FileExistsError 예외를 발생
    #           True : 폴더가 이미 존재해도 예외를 발생시키지 않고 무시

    # 해당 파일들의 리스트를 json_files 변수에 저장하는 코드
    json_files = glob.glob(os.path.join(coco_json_folder, '*.json'))

    for json_file in json_files:

      # JSON 파일을 텍스트 모드로 읽어옴
      with open(json_file, 'r') as f:
          coco_data = json.load(f)

      filename = os.path.splitext(os.path.basename(json_file))[0]
      output_txt_path = os.path.join(output_txt_folder, filename + '.txt')

      ## 읽어온 JSON 데이터를 그대로 텍스트 파일로 저장(본래 코드 - 참고용)
      ## with open('output.txt', 'w') as f:
      ##      f.write(json_data)

      # YOLO 형식의 텍스트 파일을 생성합니다.
      with open(output_txt_path, 'w') as f:

          # COCO 데이터의 각 어노테이션(annotation)을 처리합니다.
          for idx, annotation in enumerate(coco_data['annotations']):

              # 어노테이션 정보에서 이미지 ID, 파일명, 너비, 높이, 클래스 ID, 바운딩 박스 정보를 추출합니다.
              image_id = annotation['image_id']

              ## next(iterator[, default])
              ## -> iterator(반복자)에서 다음 요소를 반환하는 역할

              image_info = next(image for image in coco_data['images'] if image['id'] == image_id)
              # -> id 값이 image_id와 일치하는 첫 번째 이미지 정보를 가져옴

              image_filename = image_info['file_name']
              image_width = image_info['width']
              image_height = image_info['height']

              class_id = annotation['category_id']
              bbox = annotation['bbox']

              ## COCO 형식의 bounding box는 [x_min, y_min, width, height]
              ## [왼쪽 상단 모서리의 x 좌표, 왼쪽 상단 모서리의 y 좌표, 너비, 높이]

              x_center = bbox[0] + bbox[2] / 2    # x 좌표를 이미지의 너비에 대한 상대값으로 표현
              y_center = bbox[1] + bbox[3] / 2    # y 좌표를 이미지의 높이에 대한 상대값으로 표현
              width = bbox[2]                     # 이미지의 너비에 대한 상대값으로 표현
              height = bbox[3]                    # 이미지의 높이에 대한 상대값으로 표현

              # YOLO 형식에 맞게 바운딩 박스의 중심 좌표와 크기를 이미지의 크기로 정규화합니다.
              x_center /= image_width
              y_center /= image_height
              width /= image_width
              height /= image_height

              ## YOLO 형식의 bounding box는 [x_center, y_center, width, height]
              # YOLO 텍스트 파일에 해당 이미지의 정보를 기록합니다.
              line = f'{idx} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n'

              # 파일에 작성
              f.write(line)

In [ ]:
# COCO JSON 파일 경로
coco_json_folder = '/content/train'

# 생성할 YOLO 텍스트 파일 경로
output_txt_folder = '/content/labels'

# 함수 실행
convert_coco_to_yolo(coco_json_folder, output_txt_folder)

In [ ]:
# 2. YOLO 적용
!pip install ultralytics

In [ ]:
from ultralytics import YOLO, checks, hub
checks()

In [ ]:
# 사이트에서 복사해서 사용: 매번 바뀜
hub.login('1e0704169690ba0348e8638360e05170d81d03b5d1')

model = YOLO('https://hub.ultralytics.com/models/JqgJBT2WsjC8zSxl6h3n')
model.train()